<a href="https://colab.research.google.com/github/amsadeghzadeh/amsadeghzadeh.github.io/blob/master/AP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!python3 /content/drive/My\ Drive/carlini/nn_robust_attacks/DF/train_DF_model.py


In [0]:
import os
os.environ['PYTHONPATH'] += ":/env/python:/content/drive/My\ Drive/adversarial_patch/my_ap/"

In [8]:
!echo $PYTHONPATH

/env/python:/env/python:/content/drive/My\ Drive/adversarial_patch/my_ap/


In [0]:
!python3 drive/My\ Drive/carlini/nn_robust_attacks/my.py 

In [5]:
! pip install git+https://github.com/nottombrown/imagenet_stubs --upgrade

  

  Cloning https://github.com/nottombrown/imagenet_stubs to /tmp/pip-req-build-l9guurob
  Running setup.py bdist_wheel for imagenet-stubs ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-0i7iy785/wheels/ae/51/e7/c1cfa8692ea864410aa778b169fc7766af073ac52a1bc23301
Successfully built imagenet-stubs


In [0]:
!python3 /content/drive/My\ Drive/adversarial_patch/my_ap/WF_AP.py

In [2]:
%cd /content/drive/My\ Drive/adversarial_patch/my_ap

/content/drive/My Drive/adversarial_patch/my_ap


In [4]:
%pwd

'/content/drive/My Drive/adversarial_patch/my_ap'

In [0]:
import tensorflow as tf



import os
import os.path as osp
import PIL.Image
import time

import keras
from keras import backend as K
import numpy as np

from StubImagenet import DFDataLoader
from meta_model import MetaModel
from display import DF_report,make_adversarial_training_data
from save_load import save_obj,load_obj


from DF.model_kears import  DF_keras_Model
from train_DF_model import train


DF_NB_CLASSES = 95
TRACE_LENGTH = 5000
TRACE_MASK_LENGTH = 100

PATCH_SHAPE = (299, 299, 3)
TRACE_PATCH_SHAPE=(TRACE_LENGTH,1)
BATCH_SIZE = 64
INPUT_SHAPE = (TRACE_LENGTH,1)





MODEL_NAMES = ['DF']

TARGET_LABEL = 73


LEARNING_RATE = 20

print(MODEL_NAMES)

# Data augmentation
# Empirically found that training with a very wide scale range works well
# as a default
SCALE_MIN = 0.
SCALE_MAX = 12.


# Local data dir to write files to
DATA_DIR = 'adversarial_patch_things'

###############################################################################

#for example_image in image_loader.get_images()[:2]:
    #print("Example true image:")
    #show(example_image)
############################################################################################
####################################################################################################


# @title class ModelState()


def get_peace_mask(shape):
    path = osp.join(DATA_DIR, "peace_sign.png")
    pic = PIL.Image.open(path)
    pic = pic.resize(shape[:2], PIL.Image.ANTIALIAS)
    if path.endswith('.png'):
        ch = 4
    else:
        ch = 3
    pic = np.array(pic.getdata()).reshape(pic.size[0], pic.size[1], ch)
    pic = pic / 127.5 - 1
    pic = pic[:, :, 3]

    peace_mask = (pic + 1.0) / 2
    peace_mask = np.expand_dims(peace_mask, 2)
    peace_mask = np.broadcast_to(peace_mask, shape)
    return peace_mask



def _line_mask(shape, sharpness=40):
    """Return a circular mask of a given shape"""
    mask = np.zeros(shape,dtype=np.float32)
    mask[np.arange(0,TRACE_MASK_LENGTH)] = 1
    return mask




def _DF_gen_target_ys(target_cls = TARGET_LABEL):
    label = target_cls
    y_one_hot = np.zeros(DF_NB_CLASSES)
    y_one_hot[label] = 1.0
    y_one_hot = np.tile(y_one_hot, (BATCH_SIZE, 1))
    return y_one_hot



class ModelContainer():
    """Encapsulates an Imagenet model, and methods for interacting with it."""

    def __init__(self, model_name, verbose=True, peace_mask=None, peace_mask_overlay=0.0):
        # Peace Mask: None, "Forward", "Backward"
        self.model_name = model_name
        self.graph = tf.Graph()
        self.sess = tf.Session(graph=self.graph)
        self.peace_mask = peace_mask

        self.patch_shape = TRACE_PATCH_SHAPE
        self._peace_mask_overlay = peace_mask_overlay
        self.load_model(verbose=verbose)

    def patch(self, new_patch=None):
        """Retrieve or set the adversarial patch.

        new_patch: The new patch to set, or None to get current patch.

        Returns: Itself if it set a new patch, or the current patch."""
        if new_patch is None:
            return self._run(self._clipped_patch)

        self._run(self._assign_patch, {self._patch_placeholder: new_patch})
        return self

    def reset_patch(self):
        """Reset the adversarial patch to all zeros."""
        self.patch(np.zeros(self.patch_shape))

    def train_step(self, data=None, target_ys=None, learning_rate=LEARNING_RATE, scale=None, dropout=None,
                   patch_disguise=None, disguise_alpha=None,Target_cls=TARGET_LABEL):
        """Train the model for one step.

        Args:
          images: A batch of images to train on, it loads one if not present.
          target_ys: Onehot target vector, defaults to TARGET_ONEHOT
          learning_rate: Learning rate for this train step.
          scale: Either a scalar value for the exact scale, or a (min, max) tuple for the scale range.

        Returns: Loss on the target ys."""

        if data is None:
            data,true_label = Trace_loader.get_train_DF_Data(BATCH_SIZE,target_cls=Target_cls)
        if target_ys is None:
            target_ys = _DF_gen_target_ys(target_cls=Target_cls)
        if scale is None:
            scale = (1, 7)
        feed_dict = {self._Trace_input: np.asarray(data).reshape([BATCH_SIZE,TRACE_LENGTH,1]),
                     self._target_ys: target_ys,
                     self._learning_rate: learning_rate}

        if patch_disguise is not None:
            if disguise_alpha is None:
                raise ValueError("You need disguise_alpha")
            feed_dict[self.patch_disguise] = patch_disguise
            feed_dict[self.disguise_alpha] = disguise_alpha
        loss, grad, tp,prob,t_vec,pi = self._run([self._loss, self._train_op,self.Trace_patch,self._probabilities,self.transform_vecs,self._patched_input], feed_dict, scale=scale, dropout=dropout)
        acc = np.mean(np.equal(np.argmax(prob, 1), np.argmax(true_label, 1)))
        target_acc = np.mean(np.equal(np.argmax(prob,1),np.argmax(target_ys,1)))
        patch_volume = np.sum(np.abs(tp))
        tmp_patch = np.multiply(np.maximum(np.sign(np.multiply(tp, fxx)), 0), tp)
        training_model_to_patch = np.ceil(np.minimum(tmp_patch, 0)) + np.floor(np.maximum(tmp_patch, 0))
        preprocess_patch_volume = np.sum(np.abs(training_model_to_patch))
        data_volume = np.sum(np.abs(data)) / BATCH_SIZE
        oh = patch_volume / data_volume
        oh_pre = preprocess_patch_volume / data_volume

        return loss,acc,target_acc,np.round(oh,decimals=2),np.round(oh_pre,decimals=2)

    def inference_batch(self, data=None, target_ys=None, scale=None):
        """Report loss and label probabilities, and patched images for a batch.

        Args:
          images: A batch of images to train on, it loads if not present.
          target_ys: The target_ys for loss calculation, TARGET_ONEHOT if not present."""
        labels = []
        if data is None:
            data,labels = Trace_loader.get_test_DF_Data(BATCH_SIZE, target_cls=TARGET_LABEL)
            data = np.array(data).reshape([BATCH_SIZE, TRACE_LENGTH, 1])
        if target_ys is None:
            target_ys = _DF_gen_target_ys(target_cls=TARGET_LABEL)
        feed_dict = {self._Trace_input: data, self._target_ys: target_ys}


        loss_per_example, ps, ims,t_vec = self._run([self._loss_per_example, self._probabilities, self._patched_input,self.transform_vecs],
                                              feed_dict, scale=scale)
        return loss_per_example, ps, ims,data,labels,t_vec

    def load_model(self, verbose=True):

        keras_mode = False
        patch = None
        self.DF_INPUT_SHAPE = (TRACE_LENGTH, 1)
        self._DF_make_model_and_ops(keras_mode, patch, verbose)

    def _run(self, target, feed_dict=None, scale=None, dropout=None):
        K.set_session(self.sess)
        if feed_dict is None:
            feed_dict = {}
        feed_dict[self.learning_phase] = False

        if scale is not None:
            if isinstance(scale, (tuple, list)):
                scale_min, scale_max = scale
            else:
                scale_min, scale_max = (scale, scale)
            feed_dict[self.scale_min] = scale_min
            feed_dict[self.scale_max] = scale_max

        if dropout is not None:
            feed_dict[self.dropout] = dropout

        return self.sess.run(target, feed_dict=feed_dict)

    def _DF_make_model_and_ops(self,  keras_mode, patch_val, verbose):
        start = time.time()
        K.set_session(self.sess)
        with self.sess.graph.as_default():
            DF_keras_model = DF_keras_Model("DF/models/DF", self.sess, self.DF_INPUT_SHAPE, DF_NB_CLASSES, TRACE_LENGTH)
            M = DF_keras_model.predict
            self.learning_phase = K.learning_phase() # if 1 -> train. if 0 -> test. (must be set in test or train time )


            Trace_shape = (TRACE_LENGTH,1)
            self._Trace_input = keras.layers.Input(shape=Trace_shape) # input (set in test or train time )

            self.scale_min = tf.placeholder_with_default(SCALE_MIN, [],name='Scale_min') # ?
            self.scale_max = tf.placeholder_with_default(SCALE_MAX, [],name='scale_max') # ?
            self._scales = tf.random_uniform([BATCH_SIZE], minval=self.scale_min, maxval=self.scale_max) # ?

            Trace_input = self._Trace_input
            self.patch_disguise = tf.placeholder_with_default(tf.zeros(self.patch_shape), shape=self.patch_shape,name='patch_disguise') #loss += l2(final_patch and patch_disguise) * disguise_alpha
            self.disguise_alpha = tf.placeholder_with_default(0.0, [],name='disguise_alpha')
            self.sign_loss_beta = tf.placeholder_with_default(0.0, [], name='disguise_alpha')
            self.Trace_patch = tf.Variable(initial_value=tf.zeros(self.patch_shape), dtype=tf.float32,name="patch")
            fx = (np.ones(Trace_shape) * -1).astype(np.float32) * -1
            fx[::2] *= -1
            self.Trace_reverse_patch_acceptable_sign = tf.convert_to_tensor(fx,dtype=tf.float32,name="patch_acceptable_sign")
            self._patch_placeholder = tf.placeholder(dtype=tf.float32, shape=self.patch_shape,name='patch_placeholder')
            self._assign_patch = tf.assign(self.Trace_patch, self._patch_placeholder) # assign _patch_placeholder to patch
            self.zero = tf.zeros(self.patch_shape)



            self.modified_patch = self.Trace_patch


            def clip_to_valid_image(x):
                return tf.clip_by_value(x, clip_value_min=-5000., clip_value_max=5000.) # sure image pixels between -1 and 1

            self._clipped_patch = clip_to_valid_image(self.modified_patch) # after making patch, use this variable to get path

            self.dropout = tf.placeholder_with_default(1.0, [])
            patch_with_dropout = tf.nn.dropout(self.modified_patch, keep_prob=self.dropout)
            #patched_input = clip_to_valid_image(self._random_overlay(Trace_input, patch_with_dropout, Trace_shape))
            patched_input = clip_to_valid_image(self._Trace_random_overlay(Trace_input, patch_with_dropout, Trace_shape))

            # Since this is a return point, we do it before the Keras color shifts
            # (but after the resize, so we can see what is really going on)
            self._patched_input = patched_input

            #if keras_mode:
             #   patched_input = to_keras(patched_input)

            # Labels for our attack (e.g. always a toaster)
            self._target_ys = tf.placeholder(tf.float32, shape=(None, DF_NB_CLASSES),name="target_ys")

            #model = M(input_tensor=patched_input)

            # Pre-softmax logits of our pretrained model
            #logits = model.outputs[0].op.inputs[0]

            logits = M(patched_input)

            self._loss_per_example = tf.nn.softmax_cross_entropy_with_logits(
                labels=self._target_ys,
                logits=logits
            )
            self._target_loss = tf.reduce_mean(self._loss_per_example)

            self._patch_loss = tf.nn.l2_loss(self.Trace_patch - self.patch_disguise) * self.disguise_alpha
            self.WF_sign_loss = tf.reduce_mean(self.Trace_patch * self.Trace_reverse_patch_acceptable_sign) * self.sign_loss_beta

            self._loss = self._target_loss + self._patch_loss + self.WF_sign_loss

            # Train our attack by only training on the patch variable
            self._learning_rate = tf.placeholder(tf.float32,name='learning_rate')
            self._optimizer = tf.train.RMSPropOptimizer(self._learning_rate)
            self._train_op = self._optimizer.minimize(self._loss, var_list=[self.Trace_patch])
            #self._grad_opt = self._optimizer.compute_gradients(self._loss,var_list=[self.Trace_patch])
            #self.mask_sign_and_grad_multiply = tf.multiply(self.Trace_patch_acceptable_sign,tf.math.abs(self._grad_opt[0][1]))
            #self.mask_sign_and_grad_add = self.mask_sign_and_grad_multiply + self.Trace_patch
            #self.mask_sign_and_grad_positive = tf.maximum(0.,self.mask_sign_and_grad_multiply)
            #self.good_grad = tf.multiply(self.mask_sign_and_grad_positive,self._grad_opt)
            #self.assign = tf.assign(self.Trace_patch,self.mask_sign_and_grad_add)
            #self._train_op = self._optimizer.apply_gradients(self._grad_opt)


            self._probabilities = tf.nn.softmax(logits=logits)

            if patch_val is not None:
                self.patch(patch_val)
            else:
                self.reset_patch()

            elapsed = time.time() - start
            if verbose:
                print("Finished loading {}, took {:.0f}s".format(self.model_name, elapsed))

            self.sess.run(tf.variables_initializer(self._optimizer.variables()))


    def _pad_and_tile_patch(self, patch, image_shape):
        # Calculate the exact padding
        # Image shape req'd because it is sometimes 299 sometimes 224

        # padding is the amount of space available on either side of the centered patch
        # WARNING: This has been integer-rounded and could be off by one.
        #          See _pad_and_tile_patch for usage
        return tf.stack([patch] * BATCH_SIZE)


    def _Trace_random_overlay(self, Traces, patch, Trace_shape):
        """Augment images with random rotation, transformation.

        Image: BATCHx299x299x3
        Patch: 50x50x3

        """
        # Add padding

        self.Trace_mask = _line_mask(Trace_shape)

        self.Trace_mask = tf.stack([self.Trace_mask] * BATCH_SIZE)
        padded_patch = tf.stack([patch] * BATCH_SIZE)

        self.transform_vecs = []

        def _random_transformation(scale_min, scale_max, Trace):
            number_of_burst_in_trace_i = np.count_nonzero(Trace)
            start =  np.round(np.random.randint(low=0,high=np.maximum(1,number_of_burst_in_trace_i)))
            if(Trace[start,0] < 0):
                start += 1
            return np.array([start]).astype(np.float32)

        for i in range(BATCH_SIZE):
            # Shift and scale the patch for each image in the batch
            random_xform_vector = tf.py_func(_random_transformation, [self.scale_min, self.scale_max, Traces[i]],
                                             tf.float32)
            random_xform_vector.set_shape([1])

            self.transform_vecs.append(random_xform_vector)
        self.transform_vecs = tf.stack(self.transform_vecs)



        self.Trace_mask_scaled = []
        for i in range(BATCH_SIZE):
            # Shift and scale the patch for each image in the batch
            Trace_mask_vector = tf.roll(self.Trace_mask[i], tf.to_int32(self.transform_vecs[i][0]),axis=0)

            Trace_mask_vector.set_shape([TRACE_LENGTH,1])

            self.Trace_mask_scaled.append(Trace_mask_vector)
        self.Trace_mask_scaled = tf.stack(self.Trace_mask_scaled)



        self.padded_patch_scaled = []
        for i in range(BATCH_SIZE):
            # Shift and scale the patch for each image in the batch
            padded_patch_vector = tf.roll(padded_patch[i], tf.to_int32(self.transform_vecs[i][0]), axis=0)
            padded_patch_vector.set_shape([TRACE_LENGTH,1])

            self.padded_patch_scaled.append(padded_patch_vector)
        self.padded_patch_scaled = tf.stack(self.padded_patch_scaled)

        self.inverted_mask = (1 - self.Trace_mask_scaled)

        #return Traces * self.inverted_mask + self.padded_patch_scaled * self.Trace_mask_scaled
        return Traces + self.padded_patch_scaled * self.Trace_mask_scaled

def train_patch(model_targets_name=None,Num_of_Patch=1,Target_cls=TARGET_LABEL,Train_Steps=100):
    x = 0
    training_model_to_patch = {}
    for m in model_targets_name:
        print("Training %s" % m)
        M = MM.nc[m]
        for j in range(Num_of_Patch):
            if Target_cls == None:
                Target_claas = j
            print("Training class ",Target_claas)
            M.reset_patch()
            for i in range(Train_Steps):
                x += 1
                loss,acc,target_acc,boh,ppboh = M.train_step(scale=(1, 10),Target_cls=Target_claas)
                if i % int(Train_Steps / Train_Steps) == 0:
                    print("[%s] loss: %s acc: %s targeted_attack_success: %s ----- OH: %s PPOH: %s" % (i, loss,acc*100,target_acc*100,boh,ppboh))
            tmp_patch = np.multiply(np.maximum(np.sign(np.multiply(M.patch(),fxx)),0),M.patch())
            training_model_to_patch[j] = np.ceil(np.minimum(tmp_patch,0)) + np.floor(np.maximum(tmp_patch,0))
            #regular_training_model_to_patch[j] = M.patch()
        print("PatchLength: ", TRACE_MASK_LENGTH, " Steps:", Train_Steps, " LR: ", LEARNING_RATE)
    return training_model_to_patch

def Make_DF_Report(MC=None,Patchs=None,Target_cls=None):
    m = MC.nc['DF']
    for i in range(len(Patchs)):
        m.patch(Patchs[i])
        patch_oh = np.sum(np.abs(Patchs[i]))
        if Target_cls == None:
            Target_class = i
        DF_report(m, n=512, show_images=0, scale=5,BATCH_SIZE=BATCH_SIZE,TARGET_LABEL=Target_class,patch_oh=patch_oh)
######################@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@##############################3



Trace_loader = DFDataLoader(DF_NB_CLASSES)

print("Creating MetaModel...")
MM = MetaModel(ModelContainer = ModelContainer,MODEL_NAMES = MODEL_NAMES,PATCH_SHAPE=TRACE_PATCH_SHAPE)
model_targets = MODEL_NAMES
load_flag = 0
train_flag = 1
STEPS = 50
fxx = (np.ones([TRACE_LENGTH, 1]) * -1).astype(np.float32)
fxx[::2] *= -1
regular_training_model_to_patch = {}
if os.path.exists(osp.join(DATA_DIR, "regular_training_model_to_patch")) and load_flag == 1:
    regular_training_model_to_patch = load_obj("DF_regular_training_model_to_patch")
else:
    regular_training_model_to_patch = train_patch(model_targets_name=MODEL_NAMES,Num_of_Patch=95,Target_cls=None,Train_Steps=STEPS)
    save_obj(regular_training_model_to_patch, "DF_regular_training_model_to_patch")

report_flag=1
if report_flag:
    Make_DF_Report(MC=MM,Patchs=regular_training_model_to_patch)
AdvTrainingData_flag = 1
AdvData = {}
if AdvTrainingData_flag:
    AdvTrainingData_train,AdvTrainingLabel_train = make_adversarial_training_data(patchs=regular_training_model_to_patch,data=Trace_loader.DF_data.train_data,labels=Trace_loader.DF_data.train_labels,BATCH_SIZE=BATCH_SIZE,MC=MM,scale=5,save=1,data_type='Train1')
    AdvTrainingData_valid,AdvTrainingLabel_valid = make_adversarial_training_data(patchs=regular_training_model_to_patch,data=Trace_loader.DF_data.validation_data,labels=Trace_loader.DF_data.validation_labels,BATCH_SIZE=BATCH_SIZE,MC=MM,scale=5,save=1,data_type='Valid1')
    Trace_loader.DF_data.train_data = AdvTrainingData_train
    Trace_loader.DF_data.train_labels = AdvTrainingLabel_train
    Trace_loader.DF_data.validation_data = AdvTrainingData_valid
    Trace_loader.DF_data.validation_labels = AdvTrainingLabel_valid
else:
    print('LoadAdvData')
    Trace_loader.DF_data.train_data = np.load('AdvTrainingData_Train1.npz')['arr_0']
    Trace_loader.DF_data.train_labels = np.load('AdvTrainingLabel_Train1.npz')['arr_0']
    Trace_loader.DF_data.validation_data = np.load('AdvTrainingData_Valid1.npz')['arr_0']
    Trace_loader.DF_data.validation_labels = np.load('AdvTrainingLabel_Valid1.npz')['arr_0']


if not os.path.isdir('models'):
    os.makedirs('models')
m = MM.nc['DF']
with m.sess.graph.as_default():   
  train(Trace_loader.DF_data, "models/DF",input_shape=INPUT_SHAPE, classes=DF_NB_CLASSES, num_epochs=10)


['DF']
Loading non-defended dataset for closed-world scenario
train
valid
test
Data dimensions:
X: Training data's shape :  (76000, 5000, 1)
y: Training data's shape :  (76000, 95)
X: Validation data's shape :  (9500, 5000, 1)
y: Validation data's shape :  (9500, 95)
X: Testing data's shape :  (9500, 5000, 1)
y: Testing data's shape :  (9500, 95)
Creating MetaModel...
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Finished loading DF, took 8s
Training DF
Training class  0
[0] loss: 20.04905 acc: 93.75 targeted_attack_success: 0.0 ----- OH: 0.0 PPOH: 0.0
[1] loss: 15.566647 acc: 84.375 targeted_attack_success: 0.0 ----- OH: 0.03 PPOH: 0.0
[2] loss: 13.5111685 acc: 45.3125 targeted_attack_success: 0.0 ----- OH: 0.09 PPOH: 0.05
[3] loss: 14.830189 acc: 40.625 targeted_attack_success: 0.0 ----- OH: 0.09 PPOH: 0.05
[4] loss: 12.482303 acc: 29.6875 t